In [1]:
def unwrap_las(input_name,output_name=None):
    '''
        Функция развертывания свернутых по ширине файлов las
        
        Parameters
        -------------
        input_name : str 
            Имя исходного файла las
        
        output_name : str, optional
            Имя конечного файла las, при отсутствии в исходное добавляется приписка _unwrap
        
                        
    '''
    import pandas as pd
    
    if output_name is None:
        output_name=input_name[:-4]+'_unwrap.las'
    input_file=open(input_name)
    output_file=open(output_name,'w')
    
    
    header=0                                      #   row of the header file
    params=[]                                     #   parameters of the input las data
    stop_writing=False
    for i in input_file:                          #   header reader of the  file
        header+=1
        
        if i[0:2]=="# ":                          #   if reader reached names of the parameters in the input file
            stop_writing=True
            params.extend(i.split()[1:])
        
        if i[0:5]=='WRAP.':                       #   change WRAP. parameter to "NO" in the output file
            if i[19:21]=='NO': 
                return('The input file is not wrapped')
            i='WRAP.             NO  : Multiple lines per depth step\n'
        
        if stop_writing==False:                   #   write header to the output file
            output_file.write(i)

        if i=='~ASCII Log Data\n': break          #   if reader reached end of the header

    for i in range(len(params)):                  #   read all names of the parameters
        params[i]=params[i].rjust(10,' ')

    params_string=''.join(params)+'\n'            #   print names of the parameters to the output file
    params_string='#'+params_string[2:]
    output_file.write(params_string)
    output_file.write('~ASCII Log Data\n')
    output_file.close()
    
    
    wrap_size=len(params)//8+1                    #   how many string in one wrapped row
    row_counter=0
    full_db=[]
    values=[]
    
    for i in input_file:                          #   read each row in the input data
        row_counter+=1
        values.extend([float(q) for q in i.split()])
        if row_counter==wrap_size:
            row_counter=0
            full_db.append(values)
            values=[]
    
    DF=pd.DataFrame(full_db,columns=params)
    for i in DF.columns:                           #   for each column data aligned to right side by zeros
        DF[i]=DF[i].apply(lambda x: str(x).ljust(9,'0'))  
    
    DF.to_csv(output_name,mode='a',sep=' ',header=False,index=False)
    input_file.close()
    print('file {0} was made successfully! Great job!'.format(output_name))